In [ ]:
import glob
import torch
import numpy as np
import torch.nn as nn
import scipy.io as sio
import matplotlib.pyplot as plt
import torch.nn.functional as F
import scipy.ndimage as ndimage
from torchvision import transforms, utils
from toolz.curried import pipe, curry, compose
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import os
import pandas as pd
import matplotlib.pyplot as plt

import csv
import numpy as np


from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

import os

import scores

import argparse
import datetime
import numpy as np
import math
import os
import gc

import torch
from torch import nn, optim
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader



## Reading in the Data

In [ ]:
path_f=os.getcwd()

path_f_1=os.path.join(path_f, 'data')


names=[]
for files_txts in os.listdir(path_f_1):
    if files_txts.endswith(".csv"):
        #print(files_txts)
        names.append(files_txts)

print(names)
path_train=os.path.join(path_f_1, names[0])
path_test=os.path.join(path_f_1, names[1])

df_train=pd.read_csv(path_train)
print(df_train.shape)
print(path_train)

## Feature Engineering

In [ ]:
stab_vector=df_train['stabilityVec'].values
y=[]
for x in stab_vector:
    #print(x)
    a=np.fromstring(x[1:-1],sep=',').astype(int)
    y.append(a)
y=np.array(y) 

df_tmp = pd.DataFrame(y, columns = ['A', 'A91B', 'A82B','A73B','A64B','A55B','A46B','A37B','A28B','A19B','B'])
stab_vec_list=[ 'A91B', 'A82B','A73B','A64B','A55B','A46B','A37B','A28B','A19B']

df_train=df_train.drop("stabilityVec",axis=1) #removing the results which originally are a string
feature_cols=list(df_train)



df_train['formulaA']=df_train['formulaA_elements_Number']
df_train['formulaB']=df_train['formulaB_elements_Number']


df_train=pd.concat([df_train, df_tmp],axis=1)

y_all=df_train[stab_vec_list]
df_tmp_stable = pd.DataFrame( columns = ['Stable_compunds'])
df_tmp_stable['Stable_compunds']=np.logical_not(y_all.sum(axis=1)==0).astype(int) ## A one means it has a stable value  a 0 

df_train=pd.concat([df_train, df_tmp_stable],axis=1)
print(df_train.shape)


print(df_train.shape)


df_stable=df_train.loc[np.logical_not(y_all.sum(axis=1)==0)]
print(df_stable.shape)
df_stable.head()

In [ ]:
#X_train_new=df_stable[feature_cols] #training only on stable elements
#y_target=df_stable[stab_vec_list]
###

X_train_new=df_train[feature_cols]   #training  on all elements
y_target=df_train[stab_vec_list[0]]



In [ ]:
# Normalizing such that the magnitude is one
from sklearn.preprocessing import normalize

X_train_new_mag_1=normalize(X_train_new, axis=1) # vector magnitude is one
X_train_new_mag_1=pd.DataFrame(data=X_train_new_mag_1,columns=feature_cols)
print(X_train_new_mag_1.shape)


## Normalizing by Zscore
from scipy.stats import zscore
X_train_new_Z_score=X_train_new.apply(zscore)
print(X_train_new_Z_score.shape)



## Normalizing so that range is 0-1
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X_train_new_0_1=min_max_scaler.fit_transform(X_train_new)
X_train_new_0_1=pd.DataFrame(data=X_train_new_0_1,columns=feature_cols)
print(X_train_new_0_1.shape)


## Normalizing so that range is -1 to 1
from sklearn import preprocessing
max_abs_scaler = preprocessing.MaxAbsScaler()
X_train_new_m1_p1=max_abs_scaler.fit_transform(X_train_new)
X_train_new_m1_p1=pd.DataFrame(data=X_train_new_m1_p1,columns=feature_cols)
print(X_train_new_m1_p1.shape)


# Using PCA as input
X_train_4_PCA=df_stable[feature_cols]
indx_4_PC=X_train_4_PCA.index
X_train_new_mag_1_PCA=normalize(X_train_4_PCA, axis=1)


pca = PCA()
pca.fit(X_train_new_mag_1_PCA)
components = pca.components_[:20,:]
new_data = np.dot(X_train_new_mag_1_PCA, components.T)
X_train_new_PCA=new_data

print(X_train_new_PCA.shape)


## Taking Logarithm of High Values

X_train_new_log=X_train_new.copy()
X_train_new_log[X_train_new_log>100]=X_train_new_log[X_train_new_log>100].apply(np.log)
print(X_train_new_log.shape)

In [ ]:
y_target.head()

## Neural network

In [ ]:
import sys; sys.argv=['']; del sys
parser = argparse.ArgumentParser()

parser.add_argument_group('Optimization related arguments')
parser.add_argument('-num_epochs', default=100, type=int, help='Epochs')
parser.add_argument('-batch_size', default=20, type=int, help='Batch size')
parser.add_argument('-lr', default=1e-3, type=float, help='Learning rate')
parser.add_argument('-lr_decay_rate', default=0.9997592083, type=float, help='Decay for lr')
parser.add_argument('-min_lr', default=5e-5, type=float, help='Minimum learning rate')
parser.add_argument('-weight_init', default='xavier', choices=['xavier', 'kaiming'], help='Weight initialization strategy')
parser.add_argument('-overfit', action='store_true', help='Overfit on 5 examples, meant for debugging')
parser.add_argument('-gpuid', default=-1, type=int, help='GPU id to use')
        
parser.add_argument('-test_size', default=0.15)

parser.add_argument_group('Checkpointing related arguments')
parser.add_argument('-load_path', default='', help='Checkpoint to load path from')
parser.add_argument('-save_path', default='checkpoints/', help='Path to save checkpoints')
parser.add_argument('-save_step', default=2, type=int, help='Save checkpoint after every save_step epochs')

# ----------------------------------------------------------------------------
# input arguments and options
# ----------------------------------------------------------------------------

args = parser.parse_args()

In [ ]:
start_time = datetime.datetime.strftime(datetime.datetime.utcnow(), '%d-%b-%Y-%H:%M:%S')
if args.save_path == 'checkpoints/':
    args.save_path += start_time
    
if args.load_path != '':
    components = torch.load(args.load_path)
    model_args = components['model_args']
    model_args.gpuid = args.gpuid
    model_args.batch_size = args.batch_size

    # this is required by dataloader
    args.normalize = model_args.normalize

for arg in vars(args):
    print('{:<20}: {}'.format(arg, getattr(args, arg)))
    
# transfer all options to model
model_args = args

In [ ]:
class arg_dataset(Dataset):
    """Face landmarks Dataset"""
    
    def __init__(self, data, y_true,args):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """        
        self.data=data
        self.y_true=y_true
        self.args = args
        assert len(data) == len(y_true)
        
    
        X_train, X_test, y_train, y_test = train_test_split(data, y_true,
                                                            test_size=args.test_size,
                                                            shuffle=True,
                                                            random_state=42)
        self.data = data
        self.y_true = y_true

        self.X_train = torch.from_numpy(X_train.values).float()
        self.y_train = torch.from_numpy(y_train.values).float()
        self.X_test = torch.from_numpy(X_test.values).float()
        self.y_test = torch.from_numpy(y_test.values).float()

        self.num_data_points = {}
        self.num_data_points['train'] = len(X_train)
        self.num_data_points['test'] = len(X_test)
        
        self._split = 'train'

    @property
    def split(self):
        return self._split

    @split.setter
    def split(self, split):
        self._split = split

    # ------------------------------------------------------------------------
    # methods to override - __len__ and __getitem__ methods
    # ------------------------------------------------------------------------

    def __len__(self):
        return self.num_data_points[self._split]

    def __getitem__(self, idx):
        dtype = self._split
        item = {'index': idx}
        item['features'] = self.X_train[idx,:]
        item['outputs'] = self.y_train[idx]
        return item

    #-------------------------------------------------------------------------
    # collate function utilized by dataloader for batching
    #-------------------------------------------------------------------------

    def collate_fn(self, batch):
        dtype = self._split
        merged_batch = {key: [d[key] for d in batch] for key in batch[0]}
        out = {}
        for key in merged_batch:
            if key in {'index'}:
                out[key] = merged_batch[key]
            else:
                out[key] = torch.stack(merged_batch[key], 0)

        batch_keys = ['features', 'outputs']
        return {key: out[key] for key in batch_keys}



In [ ]:
class Network(nn.Module):
    def __init__(self, args):
        super().__init__()
        self.args = args
        self.layers = nn.Sequential(
            nn.Linear(98, 60),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(60, 30),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(30, 15),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(15, 1),
            nn.Sigmoid()
        )
    def forward(self, batch):
        return self.layers(batch)

In [ ]:
dataset = arg_dataset(X_train_new_m1_p1, y_target,args)
dataloader = DataLoader(dataset,
                        batch_size=args.batch_size,
                        shuffle=True,
                        collate_fn=dataset.collate_fn)

In [ ]:
item = dataset[0]

x = item['features'][None]
print("features: ", x)
y = item['outputs']
print("outputs: ", y)

In [ ]:
setattr(args, 'iter_per_epoch', math.ceil(dataset.num_data_points['train'] / args.batch_size))
print("{} iter per epoch.".format(args.iter_per_epoch))

In [ ]:
net = Network(model_args)
optimizer = optim.Adam(net.parameters())
criterion = nn.BCELoss()
scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=args.lr_decay_rate)

In [ ]:
# ----------------------------------------------------------------------------
# training
# ----------------------------------------------------------------------------

net.train()
os.makedirs(args.save_path, exist_ok=True)

running_loss = 0.0
train_begin = datetime.datetime.utcnow()
print("Training start time: {}".format(datetime.datetime.strftime(train_begin, '%d-%b-%Y-%H:%M:%S')))

log_loss = []
for epoch in range(1, model_args.num_epochs + 1):
    for i, batch in enumerate(dataloader):
        optimizer.zero_grad()
        for key in batch:
            batch[key] = Variable(batch[key])
            if args.gpuid >= 0:
                batch[key] = batch[key].cuda()

        # --------------------------------------------------------------------
        # forward-backward pass and optimizer step
        # --------------------------------------------------------------------
        net_out = net(batch['features'])

        cur_loss = criterion(net_out, batch['outputs'])
        cur_loss.backward()

        optimizer.step()
        gc.collect()

        # --------------------------------------------------------------------
        # update running loss and decay learning rates
        # --------------------------------------------------------------------
        #train_loss = cur_loss.data[0]
        train_loss = cur_loss.item()
        if running_loss > 0.0:
            running_loss = 0.95 * running_loss + 0.05 * train_loss
        else:
            running_loss = train_loss 

        if optimizer.param_groups[0]['lr'] > args.min_lr:
            scheduler.step()


        # --------------------------------------------------------------------
        # print after every few iterations
        # --------------------------------------------------------------------
        if i % 40 == 0:
            test_losses = []
            accuracy = []
            for i in range(int(dataset.num_data_points['test']/args.batch_size)):
                test_feat = dataset.X_test[i*args.batch_size:(i+1)*args.batch_size, :]
                test_labels = dataset.y_test[i*args.batch_size:(i+1)*args.batch_size]
                test_feat = Variable(test_feat)
                test_labels = Variable(test_labels)
                if args.gpuid >= 0:
                    test_feat = test_feat.cuda()
                    test_labels = test_labels.cuda()
                net_out = net(test_feat)
                cur_loss = criterion(net_out, test_labels)
                test_losses.append(cur_loss.item())
                
                y_pred = torch.sigmoid(net_out).data > 0.5
                y_pred = y_pred.cpu().numpy()
                accuracy.append((test_labels.cpu().numpy() == y_pred).all(axis=1))

            validation_loss = np.mean(test_losses)
            
            accuracy = np.mean(accuracy)

            iteration = (epoch - 1) * args.iter_per_epoch + i

            log_loss.append((epoch,
                             iteration,
                             running_loss,
                             train_loss,
                             validation_loss,
                             accuracy,
                             optimizer.param_groups[0]['lr']))

            # print current time, running average, learning rate, iteration, epoch
            print("[{}][Epoch: {:3d}][Iter: {:6d}][Loss: {:6f}][val loss: {:6f}][accuracy: {:6f}][lr: {:7f}]".format(
                datetime.datetime.utcnow() - train_begin, epoch,
                    iteration, running_loss, validation_loss, accuracy,
                    optimizer.param_groups[0]['lr']))

    # ------------------------------------------------------------------------
    # save checkpoints and final model
    # ------------------------------------------------------------------------
    if epoch % args.save_step == 0:
        torch.save({
            'net': net.state_dict(),
            'optimizer': optimizer.state_dict(),
            'model_args': net.args
        }, os.path.join(args.save_path, 'model_epoch_{}.pth'.format(epoch)))

torch.save({
    'net': net.state_dict(),
    'optimizer': optimizer.state_dict(),
    'model_args': net.args
}, os.path.join(args.save_path, 'model_final.pth'))

np.save(os.path.join(args.save_path, 'log_loss'), log_loss)

In [ ]:
y_pred[1]

In [ ]:
test_labels[1]